# Wikipedia Notable Life Expectancies
# [Notebook 8: Data Cleaning Part 7](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean7_thanak_2022_07_26.ipynb)
### Context

The
### Objective

The
### Data Dictionary
- Feature: Description

### Importing Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [2]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean6.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean6", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 98060 rows and 38 columns.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,None,dancer,ballet designer and director,None,None,None,None,None,None,None,None,86.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,3.091042,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,None,economist,writer,and academic,None,None,None,None,None,None,None,68.0,None,Ireland,None,None,2.564949,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
98058,9,Aamir Liaquat Hussain,", 50, Pakistani journalist and politician, MNA .",https://en.wikipedia.org/wiki/Aamir_Liaquat_Hussain,99,2022,June,", since",None,,MNA,None,None,None,None,None,None,None,None,50.0,None,Pakistan,None,"2002 2007, since 2018",4.605170,0,0,0,0,0,1,0,0,1,0,0,0,2
98059,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,None,engineer,member of the Academy of Engineering,None,None,None,None,None,None,None,None,86.0,None,"China, People's Republic of",None,None,1.386294,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
25537,4,"Richard Holme, Baron Holme of Cheltenham",", 71, British Liberal Democrat politician, cancer.","https://en.wikipedia.org/wiki/Richard_Holme,_Baron_Holme_of_Cheltenham",9,2008,May,None,None,,cancer,None,None,None,None,None,None,None,None,71.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,2.302585,0,0,0,0,0,0,0,0,1,0,0,0,1
85766,7,Fred Akers,", 82, American football coach .",https://en.wikipedia.org/wiki/Fred_Akers,5,2020,December,"Longhorns, Wyoming Cowboys, Purdue Boilermakers",None,football coach,None,None,None,None,None,None,None,None,None,82.0,None,United States of America,United States of America,"Longhorns, Wyoming Cowboys, Purdue Boilermakers",1.791759,0,0,0,0,0,0,0,0,0,0,0,0,0
32731,12,Belva Plain,", 95, American novelist .",https://en.wikipedia.org/wiki/Belva_Plain,5,2010,October,,None,novelist,None,None,None,None,None,None,None,None,None,95.0,None,United States of America,None,,1.791759,0,0,0,0,0,0,0,0,0,0,0,0,0
1956,16,"Simon Fraser, 15th Lord Lovat",", 83, British Commando during World War II and nobleman.","https://en.wikipedia.org/wiki/Simon_Fraser,_15th_Lord_Lovat",26,1995,March,None,None,Commando during World War II and nobleman,None,None,None,None,None,None,None,None,None,83.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,3.295837,0,0,0,0,0,0,0,0,0,0,0,0,0
2614,21,Jon Hinson,", 53, American politician.",https://en.wikipedia.org/wiki/Jon_Hinson,7,1995,July,None,None,,None,None,None,None,None,None,None,None,None,53.0,None,United States of America,None,None,2.079442,0,0,0,0,0,0,0,0,1,0,0,0,1


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98060 entries, 0 to 98059
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   day                        98060 non-null  object 
 1   name                       98060 non-null  object 
 2   info                       98060 non-null  object 
 3   link                       98060 non-null  object 
 4   num_references             98060 non-null  int64  
 5   year                       98060 non-null  int64  
 6   month                      98060 non-null  object 
 7   info_parenth               36661 non-null  object 
 8   info_1                     22 non-null     object 
 9   info_2                     98028 non-null  object 
 10  info_3                     48896 non-null  object 
 11  info_4                     10264 non-null  object 
 12  info_5                     1265 non-null   object 
 13  info_6                     181 non-null    obj

<IPython.core.display.Javascript object>

#### Observations:
- With our dataset loaded, we can pick up where we left off with extracting known_for values by rebuilding `known_for_dict`.

### Extracting `known_for` Continued

#### Finding `known_for` Roles in `info_2`

In [6]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [7]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [8]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "Catholic prelate" in df.loc[index, "info"]],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [9]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [10]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "defrocked" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [11]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "defrocked Catholic prelate"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [12]:
# Creating lists for each category
politics_govt_law = [
    "Patriotic",
]

arts = []
sports = []
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = [
    "Syro Malabar Catholic prelate",
    "Eastern Catholic prelate",
    "clandestine Catholic prelate",
    "Old Catholic prelate",
    "Catholic prelate and theologian",
    "Catholic prelate and first cardinal",
    "Maronite Catholic prelate",
    "Catholic prelate and Cardinal",
    "Coptic Catholic prelate",
    "Catholic prelate and bishop",
    "Catholic prelate and cardinal",
    "Catholic prelate and",
    "and Catholic prelate",
    "Catholic prelate",
]
social = []
crime = [
    "defrocked",
]
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [13]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [14]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['spiritual'] ==1].sample(2)

CPU times: total: 8.55 s
Wall time: 8.55 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
25657,21,John Aloysius Morgan,", 98, Australian Roman Catholic prelate.",https://en.wikipedia.org/wiki/John_Aloysius_Morgan,3,2008,May,None,None,,None,None,None,None,None,None,None,None,None,98.0,None,Australia,Italy,None,1.386294,0,0,1,0,0,0,0,0,0,0,0,0,1
88499,16,David Dias Pimentel,", 79, Portuguese Roman Catholic prelate, bishop of São João da Boa Vista , COVID-19.",https://en.wikipedia.org/wiki/David_Dias_Pimentel,6,2021,March,,None,,bishop of São João da Boa Vista,COVID,None,None,None,None,None,None,None,79.0,None,Portugal,Italy,2001 2016,1.945910,0,0,1,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [15]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 44902 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [16]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [17]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [18]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "physicist" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [19]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [20]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [index for index in df.index if "physicist and science" in df.loc[index, "info"]]
# ]

<IPython.core.display.Javascript object>

In [21]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "health physicist"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [22]:
# Creating lists for each category
politics_govt_law = ["arms control expert", "refusenik"]

arts = []
sports = []
sciences = [
    "physicist and group leader in the Manhattan Project",
    "physicist and molecular biologist",
    "physicist and computer research executive",
    "Nobel Prize winning biophysicist",
    "physicist and grandson of Yuan Shikai",
    "physicist and polymer researcher",
    "physicist and hydrodynamicist",
    "nuclear physicist and inventor",
    "physicist and a leader in controlled fusion research",
    "computational physicist and the father of plasma based acceleration techniques",
    "biophysicist and theoretical ecologist",
    "physicist and co winner of Nobel Prize in Physics in",
    "leading physicist in the study of waves",
    "physicist and Nobel laureate",
    "physicist ane engineer",
    "astrophysicist and radio astronomer",
    "physicist and artificial intelligence pioneer",
    "physicist and civil engineer",
    'physicist who coined the term "black hole"',
    "physicist who co discovered the Wigner Seitz cell",
    "physicist and former director of SLAC",
    "physicist at Uppsala University",
    "physicist who won the Nobel Prize for Physics in",
    "physicist who built the first laser",
    "physicist who was a pioneer of solid state physics",
    "molecular biophysicist and crystallographer",
    "physicist and member of the Manhattan Project",
    "physicist and color scientist",
    "marine geologist and geophysicist",
    "theoretical physicist and astronomer",
    "physicist and electronics engineer",
    "physicist and inventor of the first digital computer",
    "pioneering biophysicist and virologist",
    "physicist Nobel Prize in Physics laureate",
    "theoretical physicist and magneto ionic theory pioneer",
    "theoretical physicist and Nobel Prize laureate",
    "physicist and jet engine designer",
    "geophysicist and oceanographer",
    "physicist and winner of the Nobel Prize in Physics",
    "physicist known for the Casimir effect",
    "nuclear physicist who worked at the Manhattan Project Metallurgical Laboratory",
    "physicist and team member of the Manhattan Project",
    "biophysicist and biochemist",
    "experimental physicist and scientist",
    "physicist and radiation health physics pioneer",
    "physicist and co inventor of the laser with Charles Townes",
    "nuclear engineer and physicist",
    'physicist known as "the father of Pulsed Power"',
    "physicist and physical chemist",
    "chemist and nuclear physicist",
    "physicist and recipient of the Nobel Prize in Physics",
    "differential geometer and mathematical physicist",
    "physicist and statistician",
    "physicist and father of Joan Baez and Mimi Fariña",
    "nuclear physicist and engineer",
    "physicist and microbiologist",
    "nuclear physicist and ufologist",
    "chemist and biophysicist",
    "condensed matter physicist",
    "physicist and researcher",
    "geophysicist and structural geologist",
    "mesoscopic physicist",
    "physicist and specialist in solid state laser",
    "physicist and aircraft designer",
    "physicist specialized in theoretical catalysis",
    "biologist and biophysicist",
    "thermal physicist",
    "atomic physicist",
    "biophysicist and science",
    "research physicist",
    "theoretical physicist and nuclear engineer",
    "neurophysicist",
    "experimental nuclear physicist",
    "health physicist",
    "physicist and parapsychologist",
    "physicist and skeptic",
    "solid state physicist",
    "biophysicist and virologist",
    "atmospheric physicist",
    "physicist and geneticist",
    "electrical engineer and physicist",
    "climate physicist",
    "nuclear and particle physicist",
    "explosives engineer and physicist",
    "physicist and neurobiologist",
    "mathematical physicist and cosmologist",
    "metallurgist and physicist",
    "mathematical geophysicist and seismologist",
    "East physicist",
    "theoretical physicist and astrophysicist",
    "Nobel Prize winning physicist",
    "optical physicist",
    "metal physicist",
    "metal and detonation physicist",
    "solar physicist",
    "oceanographic physicist",
    "geophysicist and planetary scientist",
    "astroparticle physicist",
    "accelerator physicist",
    "engineer and physicist",
    "molecular biophysicist",
    "physicist and radio astronomer",
    "physicist and meteorologist",
    "physicist and computer scientist",
    "astronomer and physicist",
    "physicist and chemist",
    "chemical physicist",
    "physicist and electrical engineer",
    "physicist and astronomer",
    "astronomer and astrophysicist",
    "medical physicist",
    "space physicist",
    "plasma physicist",
    "chemist and physicist",
    "physicist and inventor",
    "experimental physicist",
    "physicist and engineer",
    "mathematical physicist",
    "particle physicist",
    "biophysicist",
    "geophysicist",
    "and nuclear physicist",
    "nuclear physicist and",
    "nuclear physicist",
    "and astrophysicist",
    "astrophysicist and",
    "astrophysicist",
    "and theoretical physicist",
    "theoretical physicist and",
    "theoretical physicist",
    "physicist and",
    "and physicist",
    "physicist",
]

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [23]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [24]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['sciences'] ==1].sample(2)

CPU times: total: 1min 10s
Wall time: 1min 10s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
93125,2,Inder Bir Singh Passi,", 82, Indian mathematician.",https://en.wikipedia.org/wiki/Inder_Bir_Singh_Passi,3,2021,October,None,None,,None,None,None,None,None,None,None,None,None,82.0,None,India,None,None,1.386294,1,0,0,0,0,0,0,0,0,0,0,0,1
44328,5,Kenneth John Frost,", 78, American astrophysicist.",https://en.wikipedia.org/wiki/Kenneth_John_Frost,16,2013,August,None,None,,None,None,None,None,None,None,None,None,None,78.0,None,United States of America,None,None,2.833213,1,0,0,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [25]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 44009 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [26]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [27]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [28]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "architect" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [29]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [30]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "architect and art" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [31]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "naval architect"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [32]:
# Creating lists for each category
politics_govt_law = ["asylum seeker"]

arts = [
    "architectural and interior designer",
    "photographer and architect",
    "Southwestern style architect",
    "architectural lighting designer",
    "church architect and Gothic Revival designer",
    "medieval architectural",
    "architect and organ designer",
    "landscape and garden architect",
    "set costume designer and architect",
    "architect and raconteur",
    "architect in Oregon",
    "architect of perestroika",
    "architect and acoustician",
    "architecture critic for",
    "interior designer and architect",
    "architect and historic",
    "architect and photographer",
    "furniture designer and architect",
    "architect and interior designer",
    "architect and graphic designer",
    "architect and art collector",
    "and course architect",
    "horticultural architect",
    "architect and designer of the flag of",
    "architect and furniture designer",
    "furniture designer and interior architect",
    "architect and landscape architect",
    "architecture and blues",
    "temple architect and sculptor",
    "bridge architect",
    "architectural critic",
    "industrial designer and architect",
    "architect and caveman",
    "literature and architecture",
    "space architect and spaceport planner",
    "architect and industrial designer",
    "town planner and architect",
    "architectural photographer",
    "architect and sculptor",
    "potter and architect",
    "architecture critic",
    "naval architect",
    "architect and urban designer",
    "golf course architect",
    "designer and architect",
    "architect and architectural",
    "sculptor and architect",
    "architect and town planner",
    "modernist architect",
    "architect and urban planner",
    "architect and designer",
    "landscape architect and",
    "landscape architect",
    "and architectural",
    "architectural",
    "of architecture",
    "and restoration architect",
    "architecture",
    "architect and",
    "and architect",
    "architect",
]
sports = []
sciences = [
    "computer architect and high tech",  # before arts
]

business_farming = []
academia_humanities = [
    "antique and architecture preservationist",  # before arts
]
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [33]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [34]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

CPU times: total: 34.7 s
Wall time: 34.7 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
94175,22,Baba Suwe,", 63, Nigerian actor and comedian.",https://en.wikipedia.org/wiki/Baba_Suwe,16,2021,November,None,None,,None,None,None,None,None,None,None,None,None,63.0,None,Nigeria,None,None,2.833213,0,0,0,0,0,1,0,0,0,0,0,0,1
70855,18,Robert Venturi,", 93, American architect, Pritzker Prize winner , complications from Alzheimer's disease.",https://en.wikipedia.org/wiki/Robert_Venturi,27,2018,September,,None,,Pritzker Prize winner,complications from Alzheimer disease,None,None,None,None,None,None,None,93.0,None,United States of America,None,1991,3.332205,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [35]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 43450 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [36]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [37]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [38]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "photographer" in df.loc[index, "info"]],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [39]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [40]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "photographer of the" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [41]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "public relations executive and photographer"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [42]:
# Creating lists for each category
politics_govt_law = []

arts = [
    "crime photographer",
    "photographer and illustrator",
    "adult photographer",
    "photographer of children",
    'photographer who pioneered "environmental portraiture"',
    "photographer at the fall of Saigon",
    "photographer of indigenous peoples in",
    "underwater photographer and filmmaker",
    "photographer and founder of",
    "photographer and editor",
    "photographer and camera operator",
    "underwater nature photographer",
    "photographer and news executive",
    "fashion and portrait photographer",
    "photographer based in San Francisco",
    "photographer born in Mérida",
    "photographer and secret FBI",
    "photographer during World War II",
    "glamour photographer and director of pornographic films",
    "photographer and war correspondent",
    "newspaper photographer",
    "double bassist and photographer",
    "photographer and photo essayist",
    "portrait photographer",
    "wilderness photographer",
    "press photographer",
    "music producer and photographer",
    "publisher and photographer",
    "photographer and art critic",
    "photographer and ballet dancer",
    "photographer and publicist",
    "photographer and theatre director",
    "engraver and photographer",
    "advertising photographer",
    "erotic photographer",
    "graphic designer and photographer",
    "photographer and art director",
    "public relations executive and photographer",
    "punk rock and art photographer",
    "fine art photographer",
    "photographer and documentary filmmaker",
    "environmental photographer",
    "photographer and blogger",
    "photographer and cinematographer",
    "photographer and biographer",
    "printmaker and photographer",
    "jazz and blues photographer",
    "commercial photographer",
    "aerial photographer and director",
    "newspaper and magazine photographer",
    "photographer and film maker",
    "filmmaker and photographer",
    "photographer and graphic designer",
    "street photographer",
    "photographer and model",
    "model and photographer",
    "news photographer",
    "and wildlife photographer",
    "wildlife photographer",
    "jazz photographer",
    "aerial photographer",
    "documentary photographer",
    "celebrity photographer",
    "photographer and filmmaker",
    "art photographer",
    "Pulitzer Prize winning photographer",
    "fashion photographer and",
    "fashion photographer",
    "photographer of the",
    "and Holocaust photographer",
    "photographer and",
    "and photographer",
    "photographer",
]
sports = []
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [43]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [44]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

CPU times: total: 39.4 s
Wall time: 39.4 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
41116,16,Josh Weston,", 39, American gay porn actor, AIDS-related causes.",https://en.wikipedia.org/wiki/Josh_Weston,4,2012,December,None,None,,AIDS related causes,None,None,None,None,None,None,None,None,39.0,None,United States of America,None,None,1.609438,0,0,0,0,0,1,0,0,0,0,0,0,1
21055,24,Octavia Butler,", 58, American science fiction author and MacArthur Foundation Fellow, head injury.",https://en.wikipedia.org/wiki/Octavia_Butler,92,2006,February,None,None,,head injury,None,None,None,None,None,None,None,None,58.0,None,United States of America,None,None,4.532599,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [45]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 42922 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [46]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [47]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [48]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "economist" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [49]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [50]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "health economist" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [51]:
# # Example code to quick-check a specific entry
# df[
#     df["info_2"]
#     == "economist who did pioneering research in linear programming and environmental economics"
# ]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [52]:
# Creating lists for each category
politics_govt_law = [
    "jurist and economist",
    "Marxian economist and founding editor of the",
    "economist specializing in public economics and information economics",
    "supply side economist",
    "libertarian economist",
    "monetarist and free market economist",
    "economist and government adviser",
    "economist and banking official",
    "economist and government advisor",
    "Marxian economist and a Trotskyist activist and",
    "economist and government minister",
    "economist and Nobel laureate",
    "macroeconomist",
    "monetary economist",
    "economist and government official",
    "public servant and economist",
    "labor economist",
    "economist and communist",
    "economist who did pioneering research in linear",
    "development economist and",
    "Marxist economist",
    "political scientist and economist",
    "Gandhian economist",
    "aristocrat and economist",
    "economist and PZPR activist",
    "economist and political scientist",
    "economist and politologist",
    "economist and taxpayer activist",
    "administrator and economist",
    "economist and government policy advisor",
    "economist and policy adviser",
    "economist and social activist",
    "economist and political adviser",
    "economist and lobbyist",
    "economist and laureate of the Nobel Memorial Prize in Economic Sciences",
    "political economist and activist",
    "lawyer and economist",
    "economist and Nobel Prize laureate",
    "Marxian economist",
    "development economist",
    "civil servant and economist",
    "economist and political activist",
    "feminist economist",
    "economist and public servant",
    "Nobel Prize winning economist",
    "health economist",
    "agricultural economist",
    "and political economist",
    "political economist",
    "economist and an",
    "and economist",
    "economist and",
    "economist",
]

arts = []
sports = []
sciences = [
    "home economist",  # before politics_govt_law
]

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = ["convicted embezzler"]
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [53]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [54]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['politics_govt_law'] ==1].sample(2)

CPU times: total: 30.1 s
Wall time: 30.1 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
35017,25,Paulo Renato Souza,", 65, Brazilian politician, Minister of Education , heart attack.",https://en.wikipedia.org/wiki/Paulo_Renato_Souza,10,2011,June,,None,,Minister of Education,heart attack,None,None,None,None,None,None,None,65.0,None,Brazil,None,1995 2002,2.397895,0,0,0,0,0,0,0,0,1,0,0,0,1
20208,25,Sir Frederick Corfield,", 90, British politician.",https://en.wikipedia.org/wiki/Frederick_Corfield,3,2005,August,None,None,,None,None,None,None,None,None,None,None,None,90.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,1.386294,0,0,0,0,0,0,0,0,1,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [55]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 42456 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [56]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [57]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [58]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[[index for index in df.index if "judge" in df.loc[index, "info"]], "info_2",]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [59]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [60]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "judge and legal" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [61]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "circuit judge and tabloid columnist"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [62]:
# Creating lists for each category
politics_govt_law = [
    "military judge and",
    "judge on the Supreme Court of Queensland",
    "judge on the Oregon Supreme Court",
    "senior judge for the Central District Court",
    "judge and Representative from Alabama",
    "appeals court judge",
    "High Court judge",
    "intellectual property lawyer and High Court judge",
    "judge of the ACT Supreme Court",
    "judge and hereditary peer",
    "judge and Law Lord",
    "circuit judge for the Court of Appeals for the Ninth Circuit",
    "senior judge of the District Court for the Southern District of",
    "judge and feminist",
    "judge of the District Court for the Western District of Missouri",
    "judge and public servant",
    "District Court judge",
    "Bankruptcy Court judge",
    "who was the first female Supreme Court judge",
    "State judge and prosecutor at the Nuremberg war crimes trials",
    "judge and influential patent attorney",
    "district judge overseeing desegregation in the South",
    "judge and former Lord Chief Justice",
    "judge and peer",
    "civil rights lawyer and the first female federal judge",
    "federal judge who crafted the mass settlement of asbestos lawsuits",
    "senior federal judge and the first black federal prosecutor in history",
    "judge on the Court of Appeals for the Third Circuit",
    "prominent judge sitting in highest court",
    "former chief judge of the Court of Appeals for the Third Circuit",
    "Superior Court judge who presided over the Charles Manson trial",
    "and Ohio judge for years",
    "judge and Vice Chancellor of the Supreme Court",
    "City family court judge and first female judge",
    "senior judge of the Family Division of the High Court",
    "former chief judge",
    "senior judge of the District Court for the Southern District of Alabama and judge for the Middle District of Alabama",
    "senior federal appellate judge",
    "civil rights activist and judge",
    "first female judge of",
    "judge in the",
    "judge and chairperson of the Electoral Commission",
    "judge and political activist",
    "judge and anti apartheid activist",
    "lawyer and Supreme Court judge",
    "judge and disability rights campaigner",
    "senior judge of the Court of Appeals for the Ninth Circuit",
    "jurist and judge",
    "judge and independence activist",
    "senior judge of the District Court for the District of New",
    "attorney and tribal judge",
    "judge and prosecutor",
    "judge and civil servant",
    "judge and ombudsman",
    "jurist and Supreme Court judge",
    "senior federal judge",
    "lawyer and state judge",
    "colonial official and judge",
    "judge of the High Court of and",
    "Navajo judge",
    "senior and chief judge",
    "legislator and federal judge",
    "senior circuit judge",
    "judge and legal",
    "judge and barrister",
    "judge and law lord",
    "district judge and",
    "district judge",
    "judge and life peer",
    "chief judge",
    "senior judge of the District Court for the Eastern District of",
    "attorney and judge",
    "district court judge",
    "judge and lawyer",
    "judge and jurist",
    "state judge",
    "Supreme Court judge",
    "barrister and judge",
    "senior judge",
    "lawyer and judge",
    "federal judge and",
    "federal judge",
    "circuit judge and",
    "judge and",
    "and judge",
    "judge",
]

arts = []
sports = [
    "dog show judge",  # before politics_govt_law
    "boxing judge and",
    "draughts player and judge",
]
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [63]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "sports": sports,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [64]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['politics_govt_law'] ==1].sample(2)

CPU times: total: 48.3 s
Wall time: 48.3 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
83211,29,Ayo Fasanmi,", 94, Nigerian politician, MP .",https://en.wikipedia.org/wiki/Ayo_Fasanmi,15,2020,July,since,None,,MP,None,None,None,None,None,None,None,None,94.0,None,Nigeria,None,since 1983,2.772589,0,0,0,0,0,0,0,0,1,0,0,0,1
71942,3,Toby Jessel,", 84, British politician, MP .",https://en.wikipedia.org/wiki/Toby_Jessel,4,2018,December,,None,,MP,None,None,None,None,None,None,None,None,84.0,None,United Kingdom of Great Britain and Northern Ireland,None,1970 1997,1.609438,0,0,0,0,0,0,0,0,1,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [65]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 41858 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [66]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [67]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [68]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "military officer" in df.loc[index, "info"]],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [69]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [70]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "coup leader" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [71]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "Karen military officer"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [72]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = []
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = [
    "WWII military officer",
    "military officer and intelligence official",
    "Karen military officer",
    "Air Force military officer",
    "military officer and war veteran",
    "Resistance member and military officer",
    "military officer and resistance fighter",
    "military officer and National Hero of",
    "military officer of World War I and World War II",
    "and later military officer",
    "military officer and veteran affairs",
    "CIA paramilitary officer",
    "military officer and Hero of the Union",
    "military officer and coup leader",
    "military officer and pilot",
    "and military officer",
    "military officer and",
    "military officer",
]
spiritual = []
social = []
crime = [
    "human trafficker",
]
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [73]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [74]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['law_enf_military_operator'] ==1].sample(2)

CPU times: total: 10.6 s
Wall time: 10.7 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
55257,15,Carlos Alberto Brilhante Ustra,", 83, Brazilian military officer, cancer.",https://en.wikipedia.org/wiki/Carlos_Alberto_Brilhante_Ustra,7,2015,October,None,None,,cancer,None,None,None,None,None,None,None,None,83.0,None,Brazil,None,None,2.079442,0,0,0,0,0,0,0,1,0,0,0,0,1
88162,4,Osman Erbaş,", 58, Turkish military officer, helicopter crash.",https://en.wikipedia.org/wiki/Osman_Erba%C5%9F,3,2021,March,None,None,,helicopter crash,None,None,None,None,None,None,None,None,58.0,None,Turkey,None,None,1.386294,0,0,0,0,0,0,0,1,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [75]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 41548 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [76]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [77]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [78]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "jurist" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [79]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [80]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "jurist and lecturer" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [81]:
# Creating lists for each category
politics_govt_law = [
    "federal jurist",
    "barrister and jurist",
    "jurist and legal analyst",
    "jurist and blind rights campaigner",
    "jurist and last known World War I",
    "jurist and law lord",
    "jurist and Judge of the International Criminal Tribunal for the Former",
    "jurist and Chief Judge",
    "jurist and Chief Justice",
    "jurist and statesman",
    "jurist and women rights activist",
    "jurist and first female chief justice of the North Carolina Supreme Court",
    "jurist and former Governor of Sindh province",
    "jurist and Governor General",
    "jurist and human rights advocate",
    "Virgin Islander jurist",
    "jurist and human rights activist",
    "jurist and legal",
    "civil rights lawyer and jurist",
    "jurist and magistrate",
    "civil rights activist and jurist",
    "jurist and prosecutor",
    "jurist and public servant",
    "jurist and lawyer",
    "jurist and life peer",
    "attorney and jurist",
    "lawyer and jurist",
    "and jurist",
    "jurist and",
    "jurist",
]

arts = []
sports = []
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [82]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [83]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

CPU times: total: 18.3 s
Wall time: 18.3 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
91910,8,Oleksandr Rojtburd,", 59, Ukrainian painter and installation artist, director of the Odesa Fine Arts Museum , cancer.",https://en.wikipedia.org/wiki/Oleksandr_Rojtburd,12,2021,August,,None,,director of the Odesa Fine Arts Museum,cancer,None,None,None,None,None,None,None,59.0,None,Ukraine,None,2018 2019,2.564949,0,0,0,0,0,1,0,0,0,0,0,0,1
67885,3,Anthony Lejeune,", 89, British writer.",https://en.wikipedia.org/wiki/Anthony_Lejeune,6,2018,March,None,None,,None,None,None,None,None,None,None,None,None,89.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,1.945910,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [84]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 41124 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [85]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [86]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [87]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "novelist" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [88]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [89]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "n novelist"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [90]:
# Creating lists for each category
politics_govt_law = [
    "peer and son of novelist John Buchan",  # before arts
]

arts = [
    "publisher and novelist",
    "novelist and editor",
    "novelist under pseudonyms",
    "Booker Prize winning novelist",
    "novelist whose books were popular in the s and s",
    "novelist and memoirist",
    "novelist who wrote a book shortlisted for the Booker Prize in",
    "food  novelist",
    "and mystery novelist",
    "Western novelist",
    "experimental novelist",
    "novelist and Pullizer Prize winner",
    "horror novelist and playwright",
    "young adult novelist",
    "dancer and novelist",
    "children novelist",
    "Nisei novelist and playwright",
    "novelist and critic",
    "wuxia novelist",
    "literary editor and novelist",
    "science fiction novelist",
    "editor and novelist",
    "novelist and filmmaker",
    "novelist and literary columnist",
    "adaption novelist",
    "book publisher and novelist",
    "novelist and literary agent",
    "children novelist and illustrator",
    "giallo novelist",
    "Konkani litterateur and novelist",
    "literary and jazz critic",
    "film producer and novelist",
    "novelist and essayist",
    "playwright and novelist",
    "romance novelist",
    "biographer and novelist",
    "spy novelist",
    "novelist and television producer",
    "fantasy novelist",
    "novelist and dramatist",
    "mystery novelist",
    "romantic novelist",
    "historical novelist",
    "literary critic and novelist",
    "critic and novelist",
    "novelist and biographer",
    "novelist and literary critic",
    "and crime novelist",
    "crime novelist",
    "novelist and playwright",
    "adventure novelist and",
    "n novelist",
    "and novelist",
    "novelist and",
    "novelist",
]
sports = []
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [91]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "politics_govt_law": politics_govt_law,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [92]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

CPU times: total: 31.4 s
Wall time: 31.4 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
80823,12,Jacques De Decker,", 74, Belgian writer and author.",https://en.wikipedia.org/wiki/Jacques_De_Decker,5,2020,April,None,None,,None,None,None,None,None,None,None,None,None,74.0,None,Belgium,None,None,1.791759,0,0,0,0,0,1,0,0,0,0,0,0,1
51984,10,John Fox,", 90, English composer and conductor.","https://en.wikipedia.org/wiki/John_Fox_(composer,_arranger,_conductor)",7,2015,February,None,None,,None,None,None,None,None,None,None,None,None,90.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,2.079442,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [93]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 40662 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [94]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [95]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [96]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "lawyer" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [97]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [98]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [index for index in df.index if "labor leader and lawyer" in df.loc[index, "info"]]
# ]

<IPython.core.display.Javascript object>

In [99]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "lawyer and administrator"]

<IPython.core.display.Javascript object>

In [100]:
# Dropping entry that points to game show page rather than page for individual
index = df[df["link"] == "https://en.wikipedia.org/wiki/Vivienne_Nearing"].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [101]:
# Creating lists for each category
politics_govt_law = [
    "public interest lawyer",
    "lawyer and anti Apartheid activist",
    "lawyer and Chief Justice of",
    "lawyer for left wing clients and causes",
    "lawyer and key witness in the trial of Willem Holleeder",
    "Alabama state senator and lawyer",
    "labor lawyer and women rights advocate",
    "lawyer who fought for equitable access to legal services",
    "lawyer who defended dissidents",
    "defense lawyer for Saddam Hussein",
    "lawyer and civil liberties activist",
    "lawyer and former Lord of Appeal",
    "trial lawyer and political fundraiser",
    "trial lawyer and political power broker",
    "activist and lawyer",
    "pro life activist and lawyer",
    "South lawyer",
    "lawyer and partner in Jacoby & Meyers",
    "lawyer and political insider",
    "lawyer and white nationalist",
    "First Amendment lawyer",
    "lawyer and clan chief",
    "labor lawyer",
    "jailhouse lawyer",
    "lawyer and royal commissioner",
    "immigration lawyer",
    "lawyer and founder of Amnesty International",
    "libel lawyer",
    "lawyer and first Alabama Supreme Court justice",
    "lawyer and Postmaster General",
    "lawyer and longtime advisor to Jimmy Carter",
    "lawyer and Solicitor General",
    'lawyer and "fixer" for the Chicago Mafia',
    "lawyer who co founded the National Lawyers Guild",
    "lawyer and chief legal counsel to the RNC",
    "lawyer and communications executive and ambassador",
    "prosecuting lawyer who was the first attorney in the to achieve a murder conviction with exclusively circumstantial evidence",
    "lawyer who defended pacifist Ezra Pound",
    "lawyer and movie studio chairman",
    "civil rights and human rights lawyer",
    "lawyer and political advisor",
    "divorce lawyer",
    "lawyer and official",
    "criminal defense lawyer",
    "lawyer and dissident",
    "lawyer political advisor",
    "lawyer and Associate Justice of the Supreme Court of the",
    "lawyer civil rights activist",
    "lawyer and president of the ICTR",
    "lawyer and Governor of Kansas",
    "lawyer and Governor of South",
    "lawyer at the Department of Justice",
    "lawyer and a key figure in the Watergate investigation",
    "lawyer and civil rights advocate",
    "and criminal lawyer",
    "criminal lawyer and",
    "criminal lawyer",
    "lawyer and environmental activist",
    "human rights lawyer and anti apartheid activist",
    "lawyer and civil servant",
    "lawyer and civic activist",
    "lawyer and human rights advocate",
    "defense lawyer and prosecutor",
    "lawyer and presidential adviser",
    "lawyer and parliamentary draftsman",
    "landowner and lawyer",
    "lawyer and social activist",
    "lawyer and government official",
    "Māori lawyer",
    "lawyer and community leader",
    "arbitration lawyer",
    "disability activist and lawyer",
    "lawyer and legal analyst",
    "lawyer and labor activist",
    "lawyer and eminent domain",
    "lawyer and magistrate",
    "bankruptcy lawyer",
    "pro lawyer",
    "lawyer and prisoner rights activist",
    "lawyer and anti apartheid activist",
    "lawyer and right to die campaigner",
    "lawyer and prosecutor",
    "intellectual property lawyer and",
    "intellectual property lawyer",
    "lawyer and policy adviser",
    "environmental lawyer",
    "abor lawyer",
    "human rights lawyer and life peer",
    "animal welfare lawyer",
    "LGBT rights lawyer",
    "reproductive rights activist and lawyer",
    "lawyer and political strategist",
    "lawyer and LGBT activist",
    "defense lawyer",
    "trial lawyer",
    "civil servant and lawyer",
    "lawyer and public official",
    "lawyer and activist",
    "lawyer and political activist",
    "lawyer and life peer",
    "lawyer and lobbyist",
    "lawyer and feminist",
    "civil rights lawyer and activist",
    "lawyer and women rights activist",
    "human rights activist and lawyer",
    "constitutional lawyer",
    "human rights lawyer",
    "lawyer and public servant",
    "and civil rights lawyer",
    "civil rights lawyer",
    "lawyer and civil rights activist",
    "lawyer and human rights activist",
    "tax lawyer and",
    "and Native rights lawyer",
    "anti segregation lawyer and",
    "lawyer and",
    "and lawyer",
    "lawyer",
]

arts = ["for the Grateful Dead", "to the stars"]  # before politics_govt_law
sports = []
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = ["gangland criminal", "disbarred"]
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [102]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "arts": arts,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [103]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['politics_govt_law'] ==1].sample(2)

CPU times: total: 1min 7s
Wall time: 1min 7s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
80697,8,François Luc Macosso,", 81, Congolese politician.",https://en.wikipedia.org/wiki/Fran%C3%A7ois_Luc_Macosso,16,2020,April,None,None,,None,None,None,None,None,None,None,None,None,81.0,None,"Congo, Republic of the",None,None,2.833213,0,0,0,0,0,0,0,0,1,0,0,0,1
91680,29,Robert Dove,", 82, American politician and academic, parliamentarian of the U.S. Senate .",https://en.wikipedia.org/wiki/Robert_Dove,11,2021,July,",",None,,parliamentarian of the Senate,None,None,None,None,None,None,None,None,82.0,None,United States of America,None,"1981 1987, 1995 2001",2.484907,0,0,0,1,0,0,0,0,1,0,0,0,2


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [104]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 40126 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [105]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [106]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [107]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df.index
#             if "professional wrestler" in df.loc[index, "info"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [108]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [109]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "professional wrestler and valet"]

<IPython.core.display.Javascript object>

In [110]:
# # Hard-coding cause_of_death for entry with value in info_2
# index = df[df["link"] == "https://en.wikipedia.org/wiki/Tony_Parisi_(wrestler)"].index
# df.loc[index, "cause_of_death"] = "aneurysm"

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [111]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = [
    "professional wrestler and kickboxer",
    "women professional wrestler",
    "professional wrestler known as liver transplant",
    "professional wrestler known as aneurysm",
    "professional wrestler of descent",
    'professional wrestler best known as "Hercules Hernandez" or simply just "Hercules"',
    'professional wrestler best known as "Moondog King"',
    'professional wrestler known as "The Crusher"',
    "professional wrestler and booker",
    "professional wrestler for the World Wrestling Federation",
    'Olympic judo bronze medalist and professional wrestler known as "Bad News Brown"',
    'professional wrestler known as "The Black Shadow"',
    "professional wrestler during the Great Depression era",
    "WWE professional wrestler",
    'professional wrestler known as "Biff Wellington"',
    "sumo and professional wrestler",
    "professional wrestler and talent agent",
    "professional wrestler of the s s famous for feuds with Buddy Rogers",
    "bodybuilder and professional wrestler",
    "professional wrestler and wrestling manager",
    "professional wrestler and valet",
    "professional wrestler and referee",
    "Hall of Fame professional wrestler and trainer",
    "female professional wrestler",
    "lucha libre professional wrestler",
    "professional wrestler and trainer",
    "professional wrestler and promoter",
    "professional wrestler and manager",
    "Hall of Fame professional wrestler",
    "professional wrestler known as",
    "professional wrestler and",
    "and professional wrestler",
    "professional wrestler",
]
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

In [112]:
# Hard-coding cause_of_death for entry with value in info_2
index = df[df["link"] == "https://en.wikipedia.org/wiki/Jumbo_Tsuruta"].index
df.loc[index, "cause_of_death"] = "complications from liver transplant"

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [113]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [114]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['sports'] ==1].sample(2)

CPU times: total: 18.2 s
Wall time: 18.2 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
14599,8,John Borton,", 69, American professional football player .",https://en.wikipedia.org/wiki/John_Borton,3,2002,April,"Ohio State, Cleveland Browns",None,,None,None,None,None,None,None,None,None,None,69.0,None,United States of America,None,"Ohio State, Cleveland Browns",1.386294,0,0,0,0,0,0,1,0,0,0,0,0,1
78457,1,Doug Hart,", 80, American football player .",https://en.wikipedia.org/wiki/Doug_Hart,17,2020,January,Green Bay Packers,None,,None,None,None,None,None,None,None,None,None,80.0,None,United States of America,None,Green Bay Packers,2.890372,0,0,0,0,0,0,1,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [115]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 39727 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [117]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [289]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [288]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "philosopher" in df.loc[index, "info"]],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [287]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [285]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [index for index in df.index if "religious philosopher" in df.loc[index, "info"]]
# ]

<IPython.core.display.Javascript object>

In [284]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "and humanist philosopher"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [283]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = []
sciences = []

business_farming = []
academia_humanities = [
    "philosopher and education worker in Eastern",
    "Jewish philosopher and Reform",
    "and cultural philosopher",
    "classicist and philosopher",
    "philosopher and teacher",
    "postmodernist philosopher and",
    "idealist philosopher",
    "North philosopher",
    "sinologist and philosopher",
    "philosopher and the founder of the School of Economic Science",
    "existentialist philosopher",
    "East philosopher",
    "philosopher and Vice Chancellor of Oxford University",
    "philosopher and social anthropologist",
    "educator and philosopher",
    "philosopher and professor in",
    "philosopher and professor of philosophy",
    "and philosopher of social science",
    "philosopher and professor of",
    "anthropologist and philosopher of",
    "cultural theorist and philosopher",
    "philosopher and Holocaust denier",
    "philosopher and anthropologist",
    "Islamologist and philosopher",
    "New Confucian philosopher",
    "philosopher and cultural theorist",
    "philosopher and epistemologist",
    "philosopher of language",
    "philosopher and humanist",
    "philosopher and translator",
    "educational philosopher",
    "translator and philosopher",
    "philosopher of religion",
    "anthropologist and philosopher",
    "philosopher and linguist",
    "philosopher and scholar",
    "social philosopher",
    "philosopher and professor",
    "moral philosopher",
    "philosopher of",
    "and humanist philosopher",
    "and philosopher and",
    "and philosopher",
    "philosopher and",
    "philosopher",
]
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [290]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [291]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['academia_humanities'] ==1].sample(2)

CPU times: total: 24.3 s
Wall time: 24.3 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
37437,17,Hank Nelson,", 74, Australian historian, cancer.",https://en.wikipedia.org/wiki/Hank_Nelson,4,2012,February,None,None,,cancer,None,None,None,None,None,None,None,None,74.0,None,Australia,None,None,1.609438,0,0,0,1,0,0,0,0,0,0,0,0,1
53866,27,Zvi Elpeleg,", 89, Polish-born Israeli diplomat and academic.",https://en.wikipedia.org/wiki/Zvi_Elpeleg,9,2015,June,None,None,,None,None,None,None,None,None,None,None,None,89.0,None,Poland,Israel,None,2.302585,0,0,0,1,0,0,0,0,1,0,0,0,2


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [292]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 39291 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [294]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [401]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [400]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "sculptor" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [399]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [398]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "sculptor and art" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [397]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "ice hockey player and general manager"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [395]:
# Creating lists for each category
politics_govt_law = []

arts = [
    "sculptor and puzzle creator",
    "steel sculptor",
    "sculptor and studio potter",
    "modernist sculptor",
    "abstract expressionist sculptor",
    "figure sculptor",
    "sculptor and computer art pioneer",
    "Apache sculptor",
    "portrait sculptor",
    "surrealist sculptor",
    "ceramic sculptor",
    "sculptor and metalworker",
    "sculptor and medallist",
    "monumental sculptor",
    "sculptor and art impresario",
    "ceramicist and sculptor",
    "sculptor and publisher",
    "muralist and sculptor",
    "minimalist sculptor",
    "folk sculptor",
    "sculptor and muralist",
    "portrait and bust sculptor",
    "sculptor and ceramicist",
    "sculptor and illustrator",
    "kinetic sculptor",
    "sculptor and coin designer",
    "abstract sculptor",
    "potter and sculptor",
    "sculptor and printmaker",
    "sculptor and arts",
    "sculptor and art",
    "sculptor and",
    "and sculptor",
    "sculptor",
]
sports = [
    "ice hockey player and coach in the National Hockey League",
    "ice hockey player and referee",
    "Olympic ice hockey player and sports administrator",
    "professional ice hockey player and coach",
    "ice hockey player and head coach",
    "ice hockey player and general manager",
    "ice hockey player and speed skater",
    "college ice hockey player",
    "Hall of Fame ice hockey player and official",
    "Olympic bronze medalist ice hockey player",
    "ice hockey player and manager",
    "ice hockey player and goaltending coach",
    "ice hockey player and scout",
    "Olympic silver medalist ice hockey player",
    "ice hockey player and Hall of Fame member",
    "Olympic ice hockey player and coach",
    "Hall of Fame ice hockey player and coach",
    "football and ice hockey player",
    "ice hockey player and executive",
    "Olympic silver medallist ice hockey player",
    "Olympic champion ice hockey player",
    "Olympic ice hockey player and sailor",
    "professional ice hockey player and",
    "professional ice hockey player",
    "ice hockey player and coach",
    "Hall of Fame ice hockey player",
    "Olympic ice hockey player",
    "ice hockey player and",
    "and ice hockey player",
    "ice hockey player",
]
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [396]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "sports": sports,
    "arts": arts,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [ ]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

#### Checking the Number of Rows without a First Category

In [ ]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

In [293]:
print("dunzo!")

# Sound notification when cell executes
chime.success()

dunzo!


<IPython.core.display.Javascript object>

#### Finding `known_for` Roles in `info_2`

In [ ]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

In [ ]:
# Code to check each value
roles_list.pop()

In [ ]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "writer" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

In [ ]:
# # Code to check each specific value
# specific_roles_list.pop()

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "and science writer" in df.loc[index, "info"]]]

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [ ]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "outlaw country music singer songwriter"]

#### Creating Lists for Each `known_for` Category

In [ ]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = [


]
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [ ]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

#### Extracting Category from `info_2`

In [ ]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

#### Checking the Number of Rows without a First Category

In [ ]:
#### Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Observations:
- It is time to export our dataframe and start a new notebook.

### Exporting Dataset to SQLite Database [wp_life_expect_clean7.db]()

In [ ]:
# # Exporting dataframe

# # Saving dataset in a SQLite database
# conn = sql.connect("wp_life_expect_clean7.db")
# df.to_sql("wp_life_expect_clean7", conn, index=False)

# # Chime notification when cell executes
# chime.success()

# [Proceed to Data Cleaning Part 8 ]()